In [1]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for ODIAC dataset 
collection_ffco2 = "odiac-ffco2-monthgrid-v2023"
# Name of the collection for gosat budget methane. 
collection_ch4budget = "gosat-based-ch4budget-yeargrid-v1"

In [3]:
collection_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2}").json()
collection_ch4budget = requests.get(f"{STAC_API_URL}/collections/{collection_ch4budget}").json()

In [4]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [5]:
num_items_ffco2 = get_item_count(collection_ffco2["id"])
num_items_ch4budget = get_item_count(collection_ch4budget["id"])

items_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2['id']}/items?limit={num_items_ffco2}").json()["features"]
items_ch4budget = requests.get(f"{STAC_API_URL}/collections/{collection_ch4budget['id']}/items?limit={num_items_ch4budget}").json()["features"]

In [6]:
items_ffco2 = {item["properties"]["start_datetime"][:7]: item for item in items_ffco2}
items_ch4budget = {item["properties"]["start_datetime"][:10]: item for item in items_ch4budget}

asset_ffco2 = "co2-emissions"
asset_ch4budget = "prior-total"

In [7]:
rescale_ffco2 = {"max":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["max"], "min":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["min"]}
rescale_ch4budget = {"max":items_ch4budget[list(items_ch4budget.keys())[0]]["assets"][asset_ch4budget]["raster:bands"][0]["histogram"]["max"], "min":items_ch4budget[list(items_ch4budget.keys())[0]]["assets"][asset_ch4budget]["raster:bands"][0]["histogram"]["min"]}

In [44]:
from branca.colormap import LinearColormap

# Choose a color map for displaying the first observation (event)
# Please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html
color_map = "bwr" 
colormap = LinearColormap(
    colors=['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'],
    vmin=0, vmax=1500
)
colormap.caption = 'ppm-m'

# Make a GET request to retrieve information for the 2020 tile
# 2020
january_2020_tile = requests.get(

    # Pass the collection name, the item number in the list, and its ID
    f"{RASTER_API_URL}/collections/{items_ffco2['2020-01']['collection']}/items/{items_ffco2['2020-01']['id']}/tilejson.json?"

    # Pass the asset name
    f"&assets={asset_ffco2}"

    # Pass the color formula and colormap for custom visualization
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"

    # Pass the minimum and maximum values for rescaling
    f"&rescale={rescale_ffco2['min']},{rescale_ffco2['max']}", 

# Return the response in JSON format
).json()


# Make a GET request to retrieve information for the 2000 tile
# 2000
january_2000_tile = requests.get(

    # Pass the collection name, the item number in the list, and its ID
    f"{RASTER_API_URL}/collections/{items_ffco2['2000-01']['collection']}/items/{items_ffco2['2000-01']['id']}/tilejson.json?"

    # Pass the asset name
    f"&assets={asset_ffco2}"

    # Pass the color formula and colormap for custom visualization
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"

    # Pass the minimum and maximum values for rescaling
    f"&rescale={rescale_ffco2['min']},{rescale_ffco2['max']}", 

# Return the response in JSON format
).json()

january_2019_tile = requests.get(
    f"{RASTER_API_URL}/collections/{items_ch4budget['2019-01-01']['collection']}/items/{items_ch4budget['2019-01-01']['id']}/tilejson.json?"
    f"&assets={asset_ch4budget}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_ch4budget['min']},{rescale_ch4budget['max']}", 
).json()

In [48]:
# To change the location, you can simply insert the latitude and longitude of the area of your interest in the "location=(LAT, LONG)" statement

# Set the initial zoom level and center of map for both tiles
# 'folium.plugins' allows mapping side-by-side
map_ = folium.Map(location=(34, -118), zoom_start=6)

# Define the first map layer (January 2020)
map_layer_2020 = TileLayer(
    tiles=january_2020_tile["tiles"][0], # Path to retrieve the tile
    attr="GHG", # Set the attribution
    opacity=0.8, # Adjust the transparency of the layer
)

# Add the first layer to the Dual Map
map_layer_2020.add_to(map_)


# January 2019
map_layer_2019 = TileLayer(
    tiles=january_2019_tile["tiles"][0],
    attr="GHG",
    opacity=0.7,
)
map_layer_2019.add_to(map_)

# # January 2012
# map_layer_2012 = TileLayer(
#     tiles=january_2012_tile["tiles"][0],
#     attr="GHG",
#     opacity=0.7,
# )
# map_layer_2012.add_to(map_.m2)

map_.add_child(colormap)

# Visualize the Dual Map
map_

In [35]:
!pip3 install gradio

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [37]:
import gradio as gr


# Gradioインターフェースを定義
with gr.Blocks() as demo:
    gr.HTML(map_._repr_html_())  # GradioでHTML表示

# インターフェースを実行
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.42.0, however version 4.29.0 is available, please upgrade.
--------
